In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_soom
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_diabetes
from sklearn.preprocessing import MinMaxScaler

In [2]:
device = 'cpu'

In [3]:
class Net(nn.Module):
    def __init__(self, input_size, device='cpu'):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)
        
        return x


In [4]:
X, y = load_diabetes(return_X_y = True, scaled=False)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))

torch_data = TensorDataset(torch.Tensor(X).to(device), torch.Tensor(y).to(device))
data_loader = DataLoader(torch_data, batch_size=1000)

In [15]:
model = Net(input_size = X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = pytorch_soom.LM(model.parameters(), lr=1, mu=0.001, mu_dec=0.1, model=model, use_diagonal=True, hessian_approx=False, c1=1e-4, tau=0.5, line_search_method='backtrack')

all_loss = []
patience = 0
max_patience = 10
for epoch in range(100):
    print('epoch: ', epoch, end='')
    all_loss.append(0)
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)
        # opt.update(loss)

        all_loss[epoch] += loss
    
    all_loss[epoch] /= len(data_loader)

    if epoch > 0 and all_loss[epoch-1] <= all_loss[epoch]:
        patience -= 1
    else:
        patience = max_patience
        

    print(', loss: {}'.format(all_loss[epoch].detach().cpu().numpy().item()))

    if patience <= 0:
        break

epoch:  0, loss: 0.3770187497138977
epoch:  1, loss: 0.07343915104866028
epoch:  2, loss: 0.060816552489995956
epoch:  3, loss: 0.0594768263399601
epoch:  4, loss: 0.04071584343910217
epoch:  5, loss: 0.031700097024440765
epoch:  6, loss: 0.03027387335896492
epoch:  7, loss: 0.028054429218173027
epoch:  8, loss: 0.02784362994134426
epoch:  9, loss: 0.027836067602038383
epoch:  10, loss: 0.02730955369770527
epoch:  11, loss: 0.027073441073298454
epoch:  12, loss: 0.02700047940015793
epoch:  13, loss: 0.026772085577249527
epoch:  14, loss: 0.02671966329216957
epoch:  15, loss: 0.02671007812023163
epoch:  16, loss: 0.02654813788831234
epoch:  17, loss: 0.02653021365404129
epoch:  18, loss: 0.026529084891080856
epoch:  19, loss: 0.026215026155114174
epoch:  20, loss: 0.02615160308778286
epoch:  21, loss: 0.02608029916882515
epoch:  22, loss: 0.02601245604455471
epoch:  23, loss: 0.02595224417746067
epoch:  24, loss: 0.025913314893841743
epoch:  25, loss: 0.025851812213659286
epoch:  26, lo

In [14]:
model = Net(input_size = X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = pytorch_soom.LM(model.parameters(), lr=1, mu=0.001, mu_dec=0.1, model=model, use_diagonal=False, hessian_approx=True, c1=1e-4, tau=0.5, line_search_method="backtrack", line_search_cond="wolfe")

all_loss = []
patience = 0
max_patience = 10
for epoch in range(100):
    print('epoch: ', epoch, end='')
    all_loss.append(0)
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)
        # opt.update(loss)

        all_loss[epoch] += loss
    
    all_loss[epoch] /= len(data_loader)

    if epoch > 0 and all_loss[epoch-1] <= all_loss[epoch]:
        patience -= 1
    else:
        patience = max_patience
        

    print(', loss: {}'.format(all_loss[epoch].detach().cpu().numpy().item()))

    if patience <= 0:
        break

epoch:  0, loss: 0.1409468799829483
epoch:  1, loss: 0.12833218276500702
epoch:  2, loss: 0.12513069808483124
epoch:  3, loss: 0.12357795238494873
epoch:  4, loss: 0.12207825481891632
epoch:  5, loss: 0.12062596529722214
epoch:  6, loss: 0.11782469600439072
epoch:  7, loss: 0.11645546555519104
epoch:  8, loss: 0.11512772738933563
epoch:  9, loss: 0.11260152608156204
epoch:  10, loss: 0.11009863764047623
epoch:  11, loss: 0.10767234861850739
epoch:  12, loss: 0.10652058571577072
epoch:  13, loss: 0.10438939183950424
epoch:  14, loss: 0.10236299782991409
epoch:  15, loss: 0.10049460083246231
epoch:  16, loss: 0.09701022505760193
epoch:  17, loss: 0.09013498574495316
epoch:  18, loss: 0.07604096084833145
epoch:  19, loss: 0.07247930765151978
epoch:  20, loss: 0.06897406280040741
epoch:  21, loss: 0.06559523940086365
epoch:  22, loss: 0.06395187973976135
epoch:  23, loss: 0.06237947940826416
epoch:  24, loss: 0.0616178885102272
epoch:  25, loss: 0.06088113412261009
epoch:  26, loss: 0.0601